In [ ]:
#https://www.kaggle.com/datasets/ajayrana/hymenoptera-data
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision